<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/colb5BERT_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **colB5BERT:** Inference and scoring B5

Solução Baseada no notebook do Pedro Genco

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install transformers hnswlib evaluate

In [3]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull


Cloning into 'colB5BERT'...
remote: Enumerating objects: 294, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 294 (delta 73), reused 18 (delta 18), pack-reused 196
Receiving objects: 100% (294/294), 32.73 MiB | 42.53 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/colB5BERT
Already up to date.


In [4]:
import sys
import os

import json

import hnswlib
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from torch import nn
from collections import defaultdict
from evaluate import load
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel

from torch.utils import data
from torch.utils.data import DataLoader
from transformers import BatchEncoding

from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, AdamW

import textwrap
import pickle
import h5py
import logging

from transformers import BertModel, BertTokenizer

import torch
from torch.nn.functional import cosine_similarity

from pathlib import Path
from sklearn.model_selection import train_test_split




# sys.path.append('/content/colB5BERT/python/')

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Jun 28 19:49:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [71]:
df_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test_positive.tsv', sep='\t')

print(f'Test dataset: {len(df_test)}')
# Sample data to test the pipeline
df_test = df_test.sample(n = 50)


dataset_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test.tsv', sep='\t')

print(f'Dataset test: {len(dataset_test)}')

,coditem,domain,pole,item_pt_text,id,id_divisao,texto_dividido,postive_ex
0,bfi2_47,A,0,"Posso ser indiferente, frio e distante dos outros",1008,1,Cada um com seu poder $LAUGH$ saudades < $NUMB...,0
1,bfi2_47,A,0,"Posso ser indiferente, frio e distante dos outros",1008,2,", recados por amigos , cartas , pombos correio...",0
2,bfi2_47,A,0,"Posso ser indiferente, frio e distante dos outros",1008,3,difícil $LAUGH$ de um lado uma das primeiras a...,0
3,bfi2_47,A,0,"Posso ser indiferente, frio e distante dos outros",1008,4,melhor te espera PAR muita paz e luz meu amigo...,0
4,bfi2_47,A,0,"Posso ser indiferente, frio e distante dos outros",1008,5,fale bem neste dia muito especial quero te des...,0
...,...,...,...,...,...,...,...,...
633094,STA_STR10,N,0,Fico ansioso(a)/aflito(a) quando o período de ...,999,6,"amo muito ! < $NUMBER$ < $NUMBER$ """""""" Fique p...",0
633095,STA_STR10,N,0,Fico ansioso(a)/aflito(a) quando o período de ...,999,7,pai ! Feliz aniversário ! < $NUMBER$ < $NUMBER...,0
633096,STA_STR10,N,0,Fico ansioso(a)/aflito(a) quando o período de ...,999,8,rei do $NAME$ Club de $NAME$ Oeste : $NAME$ $N...,0
633097,STA_STR10,N,0,Fico ansioso(a)/aflito(a) quando o período de ...,999,9,todo tipo de público . A realização do projeto...,0


The class `Encoder` is a subclass of PyTorch's `nn.Module`, which is used to construct neural network architectures. This class is intended to serve as an encoder which takes an input (`x`), processes it using a base model (like a pretrained transformer), and returns the resulting embeddings. Here's a breakdown of the steps:

1. `__init__(self, base_model, use_cls = False)`: The constructor method. When an object of class `Encoder` is created, it needs to be provided with a `base_model` (a pre-trained model such as BERT, GPT, etc.) and an optional parameter `use_cls`. The `use_cls` parameter is a boolean flag indicating whether to use the output embedding of the [CLS] token or to average the output embeddings of all tokens.

2. `forward(self, x)`: The method that's called during the forward pass of the model. The input `x` is expected to be a dictionary containing the input IDs of the tokens and the attention mask.

3. `embeddings = self.base_model(**x)`: This line feeds the input `x` to the `base_model` and retrieves the output. The output is a tuple where the first element is the output from the transformer's encoder, i.e., the embeddings for each token in the input sequence.

4. `if self.use_cls:`: This is a conditional statement checking whether `use_cls` was set to `True` or `False`.

    - If `use_cls` is `True`, then `embeddings = embeddings[0][:, 0, :]` selects the embeddings of the [CLS] token. In the output of transformer models, the first token is usually the [CLS] token, so this line is getting the embeddings of that token.

    - If `use_cls` is `False`, then the `else` block is executed:
        - `embeddings = embeddings[0]` : Here, it is simply getting all the embeddings from the transformer's output.
        
        - `attention_mask = x["attention_mask"]` : Extracts the attention mask from the input `x`. The attention mask is used to avoid performing attention on padding token indices.

        - `input_mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()`: The attention mask is expanded to match the dimensions of `embeddings`. The mask is also converted to float type as the embeddings are floats.
        
        - `embeddings = torch.sum(embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)`: This line calculates the sum of the embeddings of all tokens in each sequence, but only for tokens that aren't padding (as indicated by the attention mask). It then divides this sum by the number of non-padding tokens in each sequence to calculate the average. The `torch.clamp` is used to prevent division by zero in case a sequence contains only padding tokens.

5. `return embeddings`: The method finally returns the computed embeddings.

So, in summary, this module can provide two kinds of embeddings: (1) Embedding of [CLS] token when `use_cls=True`, which is commonly used for sequence-level classification tasks; and (2) Average of all the non-padding token embeddings when `use_cls=False`, providing a single vector that takes into account all the tokens in the sequence.

In [67]:
class Encoder(nn.Module):
  def __init__(self, base_model, use_cls = False):
    super().__init__()
    self.base_model = base_model
    self.use_cls = use_cls

  def forward(self, x):
    embeddings = self.base_model(**x)

    if self.use_cls:
      embeddings = embeddings[0][:, 0, :]
    else:
      embeddings = embeddings[0] #First element of model_output contains all token embeddings
      attention_mask = x["attention_mask"]
      input_mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
      embeddings = torch.sum(embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    return embeddings

In [68]:
params = {
    "use_cls": False,
    "similarity_function": "dot",
    "model_name": 'neuralmind/bert-base-portuguese-cased',
}

tokenizer = AutoTokenizer.from_pretrained(params["model_name"])
model_query = AutoModel.from_pretrained(params["model_name"])
model_doc = AutoModel.from_pretrained(params["model_name"])

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predicti

In [69]:
encoder_query = Encoder(model_query, False)
encoder_doc = Encoder(model_doc, False)


#map_location=torch.device('cpu')
encoder_query.load_state_dict(torch.load('/content/drive/MyDrive/colB5BERT/model_query_epoch_0.pt'))
encoder_doc.load_state_dict(torch.load('/content/drive/MyDrive/colB5BERT/model_doc_epoch_0.pt'))


#for k,v in ckpt.items():
#    restored_ckpt[k.replace('_orig_mod.', '')] = v

#model_query = torch.compile(model_query)
#model_query.load_state_dict(restored_ckpt)




<All keys matched successfully>

In [72]:
class QueriesDataset(Dataset):
  def __init__(self, queries, tokenizer):
    self.queries = queries
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.queries)

  def __getitem__(self, idx):
    tokenized_query = self.tokenizer(
        self.queries[idx],
        padding="max_length",
        truncation=True,
        max_length=32,
        return_tensors='pt'
    )
    output = {
        "input_ids": tokenized_query["input_ids"].squeeze(),
        "attention_mask": tokenized_query["attention_mask"].squeeze()
    }
    return output

class DocumentsDataset(Dataset):
  def __init__(self, relevant_documents,  tokenizer):
    self.relevant_documents = relevant_documents
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.relevant_documents)

  def __getitem__(self, idx):
    tokenized_doc = self.tokenizer(
        self.relevant_documents[idx],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    sample = {
        "relevant_input_ids": tokenized_doc["input_ids"].squeeze(),
        "relevant_attention_mask": tokenized_doc["attention_mask"].squeeze()
    }
    return sample


In [86]:
query_dataset_eval = QueriesDataset(queries = list(dataset_test['item_pt_text']), tokenizer = tokenizer)
doc_datasets_eval = DocumentsDataset(list(dataset_test['texto_dividido']), tokenizer)

In [87]:
eval_query_dataloader = DataLoader(query_dataset_eval, batch_size = 500, shuffle=False)
eval_doc_dataloader = DataLoader(doc_datasets_eval, batch_size = 500, shuffle=False)

In [88]:
def score_b5(eval_dataloader_query, eval_dataloader_doc, model_query, model_doc, device):

  model_query.eval()
  model_doc.eval()
  outputs_scores=[]

  for query_batch, doc_batch in tqdm(zip(eval_dataloader_query, eval_dataloader_doc), total=len(eval_dataloader_query), desc='Scoring Queries'):
    with torch.no_grad():
      query_input = {
          "input_ids": query_batch["input_ids"].to(device),
          "attention_mask": query_batch["attention_mask"].to(device)
      }
      query_emb = model_query(query_input)

      relevant_input = {
          "input_ids": doc_batch["relevant_input_ids"].to(device),
          "attention_mask": doc_batch["relevant_attention_mask"].to(device)
      }

      relevant_embeddings = model_doc(relevant_input)

      ## Split in two parts: dot product of query and relevant, and dot product of query and not rel

      sim = torch.matmul(query_emb, relevant_embeddings.T) # B, H x H, B = B, B
      #print(sim)
      scores_sim = torch.diagonal(sim, 0) # B,
      #print(scores_sim)
      outputs_scores.append(scores_sim.cpu().numpy())

  return(outputs_scores)

  #print(f"Eval loss: {loss_val / len(eval_dataloader_query)}")



In [89]:
device = "cuda" if torch.cuda.is_available() else "cpu"

import gc

torch.cuda.empty_cache()
gc.collect()

encoder_doc = encoder_doc.to(device)
encoder_query = encoder_query.to(device)

b5_scores = score_b5(eval_query_dataloader, eval_doc_dataloader, encoder_query, encoder_doc, device)

Scoring Queries:   0%|          | 0/1267 [00:00<?, ?it/s]

In [105]:
len(b5_scores)
b5_scores[0:3]

b5_scores2 = np.concatenate(b5_scores, axis=0)
b5_scores[0:3]
b5_scores2[500:502]

print(b5_scores2)
b5_scores =
len(dataset_test)

with open("/content/drive/MyDrive/colB5BERT/b5_scores2.pkl", "wb") as file:
    pickle.dump(b5_scores2 , file)


array([ 4.0000343,  6.8716364,  5.893053 , ...,  2.5232058, -1.6123338,
       -1.9377005], dtype=float32)